In [1]:
import numpy as np
import pandas as pd
import os



In [2]:

def grab_final_config():
    with open("final_state.xyz", 'r') as f:
        lines = f.readlines()
        return lines

def add_tag(lines, dir):
    energy = float(lines[1].split()[1])
    lines[1] = lines[1].replace('\n','') + ' config: ' + dir + '\n'
    return lines, energy
    
def extract_eam_energy():
    with open("log.lammps", 'r') as f:
        lines = f.readlines()
        min_ene_line = 0
        for i, line in enumerate(lines):
            if '  Energy initial, next-to-last, final =' in line:
                min_ene_line = i + 1
                break
        if not min_ene_line:
            raise Exception("minimization_energy not found")
        eline = lines[min_ene_line]
        eval = float(eline.split()[0])
    return eval

def get_num_atm(lines, atm):
    num_ox = 0
    for line in lines[2:]:
        toks = line.split()
        first_char = toks[0]
        if first_char == atm:
            num_ox += 1
    return num_ox
        
        


In [3]:
os.chdir('../DATA/')
dirs = [d for d in os.listdir('.') if os.path.isdir(d) and d not in  ['.git', '.ipynb_checkpoints' ]]
dirs = sorted(dirs)

dirs


['AlN-1',
 'AlN-2',
 'N',
 'N2',
 'N2-short',
 'N4-1',
 'N4-2',
 'N4-3',
 'NN',
 'NiAlN-1',
 'NiAlN-2',
 'NiN-1',
 'NiN-2',
 'PureAl',
 'PureAlN',
 'PureNi',
 'PureNiAl',
 'SlabAl',
 'SlabAlN-1',
 'SlabAlN-2',
 'SlabAlN2-1',
 'SlabAlN2-2',
 'SlabAlN2-s1',
 'SlabAlN2-s10',
 'SlabAlN2-s11',
 'SlabAlN2-s12',
 'SlabAlN2-s13',
 'SlabAlN2-s14',
 'SlabAlN2-s15',
 'SlabAlN2-s16',
 'SlabAlN2-s17',
 'SlabAlN2-s18',
 'SlabAlN2-s19',
 'SlabAlN2-s2',
 'SlabAlN2-s20',
 'SlabAlN2-s21',
 'SlabAlN2-s22',
 'SlabAlN2-s23',
 'SlabAlN2-s24',
 'SlabAlN2-s3',
 'SlabAlN2-s4',
 'SlabAlN2-s5',
 'SlabAlN2-s6',
 'SlabAlN2-s7',
 'SlabAlN2-s8',
 'SlabAlN2-s9',
 'SlabNi',
 'SlabNiN-1',
 'SlabNiN-2',
 'SlabNiN2-1',
 'SlabNiN2-2',
 'SlabNiN2-p1',
 'SlabNiN2-s01',
 'SlabNiN2-s02',
 'SlabNiN2-s03',
 'SlabNiN2-s04',
 'SlabNiN2-s05',
 'SlabNiN2-s06',
 'SlabNiN2-s07',
 'SlabNiN2-s08',
 'SlabNiN2-s11',
 'SlabNiN2-s12',
 'SlabNiN2-s13',
 'SlabNiN2-s15',
 'SlabNiN2-s16',
 'SlabNiN2-s17',
 'SlabNiN2-s18',
 'SlabNiN2-s21',
 'Sl

In [4]:

all_configs = []
eam_energies  = []
dft_energies = []
oxygens = []
nickels = []
Als  = []
for dir in dirs:
    os.chdir(dir)
    print(dir)
    config_lines = grab_final_config()
    tagged_lines, dft_energy = add_tag(config_lines, dir)
    num_ox = get_num_atm(tagged_lines, "N")
    num_ni = get_num_atm(tagged_lines, "Ni")
    num_al = get_num_atm(tagged_lines, "Al")
    oxygens.append(num_ox)
    nickels.append(num_ni)
    Als.append(num_al)
    dft_energies.append(dft_energy)
    all_configs += tagged_lines
    os.chdir('./EAM_energy')
    try:
        eam_energies.append(extract_eam_energy()) 
    except:
        print(f"dir {dir} is likely an N only dir")
        eam_energies.append(0)
    os.chdir('../../')




AlN-1
AlN-2
N
dir N is likely an O only dir
N2
dir N2 is likely an O only dir
N2-short
dir N2-short is likely an O only dir
N4-1
dir N4-1 is likely an O only dir
N4-2
dir N4-2 is likely an O only dir
N4-3
dir N4-3 is likely an O only dir
NN
dir NN is likely an O only dir
NiAlN-1
NiAlN-2
NiN-1
NiN-2
PureAl
PureAlN
PureNi
PureNiAl
SlabAl
SlabAlN-1
SlabAlN-2
SlabAlN2-1
SlabAlN2-2
SlabAlN2-s1
SlabAlN2-s10
SlabAlN2-s11
SlabAlN2-s12
SlabAlN2-s13
SlabAlN2-s14
SlabAlN2-s15
SlabAlN2-s16
SlabAlN2-s17
SlabAlN2-s18
SlabAlN2-s19
SlabAlN2-s2
SlabAlN2-s20
SlabAlN2-s21
SlabAlN2-s22
SlabAlN2-s23
SlabAlN2-s24
SlabAlN2-s3
SlabAlN2-s4
SlabAlN2-s5
SlabAlN2-s6
SlabAlN2-s7
SlabAlN2-s8
SlabAlN2-s9
SlabNi
SlabNiN-1
SlabNiN-2
SlabNiN2-1
SlabNiN2-2
SlabNiN2-p1
SlabNiN2-s01
SlabNiN2-s02
SlabNiN2-s03
SlabNiN2-s04
SlabNiN2-s05
SlabNiN2-s06
SlabNiN2-s07
SlabNiN2-s08
SlabNiN2-s11
SlabNiN2-s12
SlabNiN2-s13
SlabNiN2-s15
SlabNiN2-s16
SlabNiN2-s17
SlabNiN2-s18
SlabNiN2-s21
SlabNiN2-s22
SlabNiN2-s23
SlabNiN2-s24
SlabNiN2-

In [9]:
references = {'AlN-1': "PureAl",
 'AlN-2':'PureAl',
 'N': None,
 'N2':None,
 'N2-short': None,
 'N4-1':None,
 'N4-2':None,
 'N4-3': None,
 'NN':None,
 'NiAlN-1': 'PureNiAl',
 'NiAlN-2': 'PureNiAl',
 'NiN-1': 'PureNi',
 'NiN-2': 'PureNi',
 'PureAl': None,
 'PureAlN': 'PureAl',
 'PureNi': None,
 'PureNiAl': None,
 'SlabAl': None,
 'SlabAlN-1': 'SlabAl',
 'SlabAlN-2': 'SlabAl',
 'SlabAlN2-1': 'SlabAl',
 'SlabAlN2-2': 'SlabAl',
 'SlabAlN2-s1': 'SlabAl',
 'SlabAlN2-s10': 'SlabAl',
 'SlabAlN2-s11': 'SlabAl',
 'SlabAlN2-s12': 'SlabAl',
 'SlabAlN2-s13': 'SlabAl',
 'SlabAlN2-s14': 'SlabAl',
 'SlabAlN2-s15': 'SlabAl',
 'SlabAlN2-s16': 'SlabAl',
 'SlabAlN2-s17': 'SlabAl',
 'SlabAlN2-s18': 'SlabAl',
 'SlabAlN2-s19': 'SlabAl',
 'SlabAlN2-s2': 'SlabAl',
 'SlabAlN2-s20':'SlabAl',
 'SlabAlN2-s21': 'SlabAl',
 'SlabAlN2-s22': 'SlabAl',
 'SlabAlN2-s23': 'SlabAl',
 'SlabAlN2-s24': 'SlabAl',
 'SlabAlN2-s3': 'SlabAl',
 'SlabAlN2-s4': 'SlabAl',
 'SlabAlN2-s5': 'SlabAl',
 'SlabAlN2-s6': 'SlabAl',
 'SlabAlN2-s7': 'SlabAl',
 'SlabAlN2-s8': 'SlabAl',
 'SlabAlN2-s9': 'SlabAl',
 'SlabNi': None,
 'SlabNiN-1':'SlabNi',
 'SlabNiN-2':'SlabNi',
 'SlabNiN2-1':'SlabNi',
 'SlabNiN2-2':'SlabNi',
 'SlabNiN2-p1':'SlabNi',
 'SlabNiN2-s01':'SlabNi',
 'SlabNiN2-s02':'SlabNi',
 'SlabNiN2-s03':'SlabNi',
 'SlabNiN2-s04':'SlabNi',
 'SlabNiN2-s05':'SlabNi',
 'SlabNiN2-s06':'SlabNi',
 'SlabNiN2-s07':'SlabNi',
 'SlabNiN2-s08':'SlabNi',
 'SlabNiN2-s11':'SlabNi',
 'SlabNiN2-s12':'SlabNi',
 'SlabNiN2-s13':'SlabNi',
 'SlabNiN2-s15':'SlabNi',
 'SlabNiN2-s16':'SlabNi',
 'SlabNiN2-s17':'SlabNi',
 'SlabNiN2-s18':'SlabNi',
 'SlabNiN2-s21':'SlabNi',
 'SlabNiN2-s22':'SlabNi',
 'SlabNiN2-s23':'SlabNi',
 'SlabNiN2-s24':'SlabNi',
 'SlabNiN2-s25':  'SlabNi',
 'SlabNiN2-s26':  'SlabNi',
 'SlabNiN2-s27':  'SlabNi',
 'SlabNiN2-s28':  'SlabNi'
             }

In [10]:
dirs

['AlN-1',
 'AlN-2',
 'N',
 'N2',
 'N2-short',
 'N4-1',
 'N4-2',
 'N4-3',
 'NN',
 'NiAlN-1',
 'NiAlN-2',
 'NiN-1',
 'NiN-2',
 'PureAl',
 'PureAlN',
 'PureNi',
 'PureNiAl',
 'SlabAl',
 'SlabAlN-1',
 'SlabAlN-2',
 'SlabAlN2-1',
 'SlabAlN2-2',
 'SlabAlN2-s1',
 'SlabAlN2-s10',
 'SlabAlN2-s11',
 'SlabAlN2-s12',
 'SlabAlN2-s13',
 'SlabAlN2-s14',
 'SlabAlN2-s15',
 'SlabAlN2-s16',
 'SlabAlN2-s17',
 'SlabAlN2-s18',
 'SlabAlN2-s19',
 'SlabAlN2-s2',
 'SlabAlN2-s20',
 'SlabAlN2-s21',
 'SlabAlN2-s22',
 'SlabAlN2-s23',
 'SlabAlN2-s24',
 'SlabAlN2-s3',
 'SlabAlN2-s4',
 'SlabAlN2-s5',
 'SlabAlN2-s6',
 'SlabAlN2-s7',
 'SlabAlN2-s8',
 'SlabAlN2-s9',
 'SlabNi',
 'SlabNiN-1',
 'SlabNiN-2',
 'SlabNiN2-1',
 'SlabNiN2-2',
 'SlabNiN2-p1',
 'SlabNiN2-s01',
 'SlabNiN2-s02',
 'SlabNiN2-s03',
 'SlabNiN2-s04',
 'SlabNiN2-s05',
 'SlabNiN2-s06',
 'SlabNiN2-s07',
 'SlabNiN2-s08',
 'SlabNiN2-s11',
 'SlabNiN2-s12',
 'SlabNiN2-s13',
 'SlabNiN2-s15',
 'SlabNiN2-s16',
 'SlabNiN2-s17',
 'SlabNiN2-s18',
 'SlabNiN2-s21',
 'Sl

In [11]:
df = pd.DataFrame({"Species": dirs,  "E_conf_DFT":dft_energies, "E_conf_EAM": eam_energies, "N_O":oxygens, "N_Ni":nickels, "N_Al": Als})
df = df.set_index("Species")
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)


              E_conf_DFT  E_conf_EAM  N_O  N_Ni  N_Al
Species                                              
AlN-1           -969.413    -860.118    1     0   256
AlN-2           -969.415    -859.881    1     0   256
N                 -0.121       0.000    1     0     0
N2               -16.489       0.000    2     0     0
N2-short         -10.147       0.000    2     0     0
N4-1             -20.387       0.000    4     0     0
N4-2             -15.955       0.000    4     0     0
N4-3             -31.099       0.000    4     0     0
NN               -78.584       0.000   10     0     0
NiAlN-1        -1314.924   -1120.123    1   125   125
NiAlN-2        -1315.040   -1120.151    1   125   125
NiN-1          -1363.928   -1136.191    1   256     0
NiN-2          -1363.160   -1135.650    1   256     0
PureAl          -961.030    -860.160    0     0   256
PureAlN          -29.938      -6.483    2     0     2
PureNi         -1355.330   -1136.474    0   256     0
PureNiAl       -1308.227   -

In [13]:
E_ox = np.zeros(len(oxygens)) + df["E_conf_DFT"]["N"]
E_ox

array([-0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093, -0.12093, -0.12093, -0.12093,
       -0.12093, -0.12093, -0.12093])

In [14]:
df["E_ox"] = E_ox

In [15]:
for i in df["E_conf_DFT"].index.values:
    try:
        print(i, references[i])
    except KeyError:
        print(i, "no_ref")

AlN-1 PureAl
AlN-2 PureAl
N None
N2 None
N2-short None
N4-1 None
N4-2 None
N4-3 None
NN None
NiAlN-1 PureNiAl
NiAlN-2 PureNiAl
NiN-1 PureNi
NiN-2 PureNi
PureAl None
PureAlN PureAl
PureNi None
PureNiAl None
SlabAl None
SlabAlN-1 SlabAl
SlabAlN-2 SlabAl
SlabAlN2-1 SlabAl
SlabAlN2-2 SlabAl
SlabAlN2-s1 SlabAl
SlabAlN2-s10 SlabAl
SlabAlN2-s11 SlabAl
SlabAlN2-s12 SlabAl
SlabAlN2-s13 SlabAl
SlabAlN2-s14 SlabAl
SlabAlN2-s15 SlabAl
SlabAlN2-s16 SlabAl
SlabAlN2-s17 SlabAl
SlabAlN2-s18 SlabAl
SlabAlN2-s19 SlabAl
SlabAlN2-s2 SlabAl
SlabAlN2-s20 SlabAl
SlabAlN2-s21 SlabAl
SlabAlN2-s22 SlabAl
SlabAlN2-s23 SlabAl
SlabAlN2-s24 SlabAl
SlabAlN2-s3 SlabAl
SlabAlN2-s4 SlabAl
SlabAlN2-s5 SlabAl
SlabAlN2-s6 SlabAl
SlabAlN2-s7 SlabAl
SlabAlN2-s8 SlabAl
SlabAlN2-s9 SlabAl
SlabNi None
SlabNiN-1 SlabNi
SlabNiN-2 SlabNi
SlabNiN2-1 SlabNi
SlabNiN2-2 SlabNi
SlabNiN2-p1 SlabNi
SlabNiN2-s01 SlabNi
SlabNiN2-s02 SlabNi
SlabNiN2-s03 SlabNi
SlabNiN2-s04 SlabNi
SlabNiN2-s05 SlabNi
SlabNiN2-s06 SlabNi
SlabNiN2-s07 SlabNi


In [16]:
erefs_dft = []
erefs_eam = []
n_ref_O = []
n_ref_Ni = []
n_ref_Al = []
ref_structs = []
for struct in dirs:
    print(struct)

    try:
        e_ref_dft = df["E_conf_DFT"][references[struct]]
    except:
        e_ref_dft = np.nan
    # print(e_ref_dft)
    try:
        e_ref_eam = df["E_conf_EAM"][references[struct]]
    except:
        e_ref_eam = np.nan
    # print(e_ref_eam)
    try:
        n_ref_ox = df["N_O"][references[struct]]
    except:
        n_ref_ox = np.nan
    try:
        n_ref_al = df["N_Al"][references[struct]]
    except:
        n_ref_al = np.nan
    try:
        n_ref_ni = df["N_Ni"][references[struct]]
    except:
        n_ref_ni = np.nan
        
    #print(e_ref_eam)
    erefs_dft.append(e_ref_dft)
    erefs_eam.append(e_ref_eam)
    n_ref_O.append(n_ref_ox)
    n_ref_Ni.append(n_ref_ni)
    n_ref_Al.append(n_ref_al)

    
    ref_struct = references[struct]
        
    ref_structs.append(ref_struct)

AlN-1
AlN-2
N
N2
N2-short
N4-1
N4-2
N4-3
NN
NiAlN-1
NiAlN-2
NiN-1
NiN-2
PureAl
PureAlN
PureNi
PureNiAl
SlabAl
SlabAlN-1
SlabAlN-2
SlabAlN2-1
SlabAlN2-2
SlabAlN2-s1
SlabAlN2-s10
SlabAlN2-s11
SlabAlN2-s12
SlabAlN2-s13
SlabAlN2-s14
SlabAlN2-s15
SlabAlN2-s16
SlabAlN2-s17
SlabAlN2-s18
SlabAlN2-s19
SlabAlN2-s2
SlabAlN2-s20
SlabAlN2-s21
SlabAlN2-s22
SlabAlN2-s23
SlabAlN2-s24
SlabAlN2-s3
SlabAlN2-s4
SlabAlN2-s5
SlabAlN2-s6
SlabAlN2-s7
SlabAlN2-s8
SlabAlN2-s9
SlabNi
SlabNiN-1
SlabNiN-2
SlabNiN2-1
SlabNiN2-2
SlabNiN2-p1
SlabNiN2-s01
SlabNiN2-s02
SlabNiN2-s03
SlabNiN2-s04
SlabNiN2-s05
SlabNiN2-s06
SlabNiN2-s07
SlabNiN2-s08
SlabNiN2-s11
SlabNiN2-s12
SlabNiN2-s13
SlabNiN2-s15
SlabNiN2-s16
SlabNiN2-s17
SlabNiN2-s18
SlabNiN2-s21
SlabNiN2-s22
SlabNiN2-s23
SlabNiN2-s24
SlabNiN2-s25
SlabNiN2-s26
SlabNiN2-s27
SlabNiN2-s28


In [17]:
df["ref_struct"] = ref_structs
df["E_ref_DFT"] = erefs_dft
df["E_ref_EAM"] = erefs_eam

In [18]:
df

,E_conf_DFT,E_conf_EAM,N_O,N_Ni,N_Al,E_ox,ref_struct,E_ref_DFT,E_ref_EAM
Species,,,,,,,,,
AlN-1,-969.413249,-860.117536,1,0,256,-0.12093,PureAl,-961.029624,-860.159853
AlN-2,-969.415355,-859.880676,1,0,256,-0.12093,PureAl,-961.029624,-860.159853
N,-0.120930,0.000000,1,0,0,-0.12093,None,NaN,NaN
N2,-16.488954,0.000000,2,0,0,-0.12093,None,NaN,NaN
N2-short,-10.147185,0.000000,2,0,0,-0.12093,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...
SlabNiN2-s24,-1304.483662,-1076.802683,2,252,0,-0.12093,SlabNi,-1288.186135,-1076.802683
SlabNiN2-s25,-1304.689408,-1076.802683,2,252,0,-0.12093,SlabNi,-1288.186135,-1076.802683
SlabNiN2-s26,-1304.782913,-1076.802683,2,252,0,-0.12093,SlabNi,-1288.186135,-1076.802683


In [19]:
df["N_O_ref"] = n_ref_O
df["N_Al_ref"] = n_ref_Al
df["N_Ni_ref"] = n_ref_Ni
df["N_Ni_ref"] = n_ref_Ni


In [20]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

              E_conf_DFT  E_conf_EAM  N_O  N_Ni  N_Al   E_ox ref_struct  \
Species                                                                   
AlN-1           -969.413    -860.118    1     0   256 -0.121     PureAl   
AlN-2           -969.415    -859.881    1     0   256 -0.121     PureAl   
N                 -0.121       0.000    1     0     0 -0.121       None   
N2               -16.489       0.000    2     0     0 -0.121       None   
N2-short         -10.147       0.000    2     0     0 -0.121       None   
N4-1             -20.387       0.000    4     0     0 -0.121       None   
N4-2             -15.955       0.000    4     0     0 -0.121       None   
N4-3             -31.099       0.000    4     0     0 -0.121       None   
NN               -78.584       0.000   10     0     0 -0.121       None   
NiAlN-1        -1314.924   -1120.123    1   125   125 -0.121   PureNiAl   
NiAlN-2        -1315.040   -1120.151    1   125   125 -0.121   PureNiAl   
NiN-1          -1363.928 

In [21]:
columns = ['E_conf_DFT', 'E_conf_EAM', 'ref_struct',  'E_ref_DFT', 
       'E_ref_EAM', 'E_ox',  'N_O', 'N_O_ref', 'N_Ni', 'N_Ni_ref', 'N_Al',  'N_Al_ref']

In [22]:
df_resort = df[columns]

In [23]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref
Species,,,,,,,,,,,,
AlN-1,-969.413249,-860.117536,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0
AlN-2,-969.415355,-859.880676,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0
N,-0.120930,0.000000,None,NaN,NaN,-0.12093,1,NaN,0,NaN,0,NaN
N2,-16.488954,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN
N2-short,-10.147185,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiN2-s24,-1304.483662,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0
SlabNiN2-s25,-1304.689408,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0
SlabNiN2-s26,-1304.782913,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0


In [24]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref
Species,,,,,,,,,,,,
AlN-1,-969.413249,-860.117536,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0
AlN-2,-969.415355,-859.880676,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0
N,-0.120930,0.000000,None,NaN,NaN,-0.12093,1,NaN,0,NaN,0,NaN
N2,-16.488954,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN
N2-short,-10.147185,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiN2-s24,-1304.483662,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0
SlabNiN2-s25,-1304.689408,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0
SlabNiN2-s26,-1304.782913,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0


In [25]:
df_resort["Al_modifier"] = df_resort["N_Al"] / df_resort["N_Al_ref"]
df_resort["Ni_modifier"] = df_resort["N_Ni"] /df_resort["N_Ni_ref"]

/tmp/ipykernel_39994/2420490869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Al_modifier"] = df_resort["N_Al"] / df_resort["N_Al_ref"]
/tmp/ipykernel_39994/2420490869.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Ni_modifier"] = df_resort["N_Ni"] /df_resort["N_Ni_ref"]


In [26]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

              E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM   E_ox  \
Species                                                                        
AlN-1           -969.413    -860.118     PureAl   -961.030   -860.160 -0.121   
AlN-2           -969.415    -859.881     PureAl   -961.030   -860.160 -0.121   
N                 -0.121       0.000       None        NaN        NaN -0.121   
N2               -16.489       0.000       None        NaN        NaN -0.121   
N2-short         -10.147       0.000       None        NaN        NaN -0.121   
N4-1             -20.387       0.000       None        NaN        NaN -0.121   
N4-2             -15.955       0.000       None        NaN        NaN -0.121   
N4-3             -31.099       0.000       None        NaN        NaN -0.121   
NN               -78.584       0.000       None        NaN        NaN -0.121   
NiAlN-1        -1314.924   -1120.123   PureNiAl  -1308.227  -1120.947 -0.121   
NiAlN-2        -1315.040   -1120.151   P

In [27]:
# df_resort["V"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["E_conf_EAM"] + df_resort["E_ref_EAM"] - (df_resort["N_O"] * df_resort["E_ox"])  

In [28]:
df_resort["E_i_DFT"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["N_O"]* df_resort["E_ox"]

/tmp/ipykernel_39994/2547235197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["E_i_DFT"] = df_resort["E_conf_DFT"] - df_resort["E_ref_DFT"] - df_resort["N_O"]* df_resort["E_ox"]


In [29]:
df_resort["E_i_EAM"] = df_resort["E_conf_EAM"] - df_resort["E_ref_EAM"]

/tmp/ipykernel_39994/2884512994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["E_i_EAM"] = df_resort["E_conf_EAM"] - df_resort["E_ref_EAM"]


In [30]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

              E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM   E_ox  \
Species                                                                        
AlN-1           -969.413    -860.118     PureAl   -961.030   -860.160 -0.121   
AlN-2           -969.415    -859.881     PureAl   -961.030   -860.160 -0.121   
N                 -0.121       0.000       None        NaN        NaN -0.121   
N2               -16.489       0.000       None        NaN        NaN -0.121   
N2-short         -10.147       0.000       None        NaN        NaN -0.121   
N4-1             -20.387       0.000       None        NaN        NaN -0.121   
N4-2             -15.955       0.000       None        NaN        NaN -0.121   
N4-3             -31.099       0.000       None        NaN        NaN -0.121   
NN               -78.584       0.000       None        NaN        NaN -0.121   
NiAlN-1        -1314.924   -1120.123   PureNiAl  -1308.227  -1120.947 -0.121   
NiAlN-2        -1315.040   -1120.151   P

In [31]:
df_resort

,E_conf_DFT,E_conf_EAM,ref_struct,E_ref_DFT,E_ref_EAM,E_ox,N_O,N_O_ref,N_Ni,N_Ni_ref,N_Al,N_Al_ref,Al_modifier,Ni_modifier,E_i_DFT,E_i_EAM
Species,,,,,,,,,,,,,,,,
AlN-1,-969.413249,-860.117536,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0,1.0,NaN,-8.262695,4.231704e-02
AlN-2,-969.415355,-859.880676,PureAl,-961.029624,-860.159853,-0.12093,1,0.0,0,0.0,256,256.0,1.0,NaN,-8.264801,2.791772e-01
N,-0.120930,0.000000,None,NaN,NaN,-0.12093,1,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
N2,-16.488954,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
N2-short,-10.147185,0.000000,None,NaN,NaN,-0.12093,2,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SlabNiN2-s24,-1304.483662,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0,NaN,1.0,-16.055667,1.998615e-10
SlabNiN2-s25,-1304.689408,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0,NaN,1.0,-16.261413,1.998615e-10
SlabNiN2-s26,-1304.782913,-1076.802683,SlabNi,-1288.186135,-1076.802683,-0.12093,2,0.0,252,252.0,0,0.0,NaN,1.0,-16.354918,1.998615e-10


In [32]:
df_resort["Al_modifier"] = df_resort["Al_modifier"].fillna(1.0)
df_resort["Ni_modifier"] = df_resort["Ni_modifier"].fillna(1.0)


/tmp/ipykernel_39994/1492507747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Al_modifier"] = df_resort["Al_modifier"].fillna(1.0)
/tmp/ipykernel_39994/1492507747.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["Ni_modifier"] = df_resort["Ni_modifier"].fillna(1.0)


In [33]:
df_resort["modifier"]  = df_resort["Al_modifier"]*df_resort["Ni_modifier"]


/tmp/ipykernel_39994/4044517486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["modifier"]  = df_resort["Al_modifier"]*df_resort["Ni_modifier"]


In [34]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

              E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM   E_ox  \
Species                                                                        
AlN-1           -969.413    -860.118     PureAl   -961.030   -860.160 -0.121   
AlN-2           -969.415    -859.881     PureAl   -961.030   -860.160 -0.121   
N                 -0.121       0.000       None        NaN        NaN -0.121   
N2               -16.489       0.000       None        NaN        NaN -0.121   
N2-short         -10.147       0.000       None        NaN        NaN -0.121   
N4-1             -20.387       0.000       None        NaN        NaN -0.121   
N4-2             -15.955       0.000       None        NaN        NaN -0.121   
N4-3             -31.099       0.000       None        NaN        NaN -0.121   
NN               -78.584       0.000       None        NaN        NaN -0.121   
NiAlN-1        -1314.924   -1120.123   PureNiAl  -1308.227  -1120.947 -0.121   
NiAlN-2        -1315.040   -1120.151   P

In [35]:
df_resort.columns

Index(['E_conf_DFT', 'E_conf_EAM', 'ref_struct', 'E_ref_DFT', 'E_ref_EAM',
       'E_ox', 'N_O', 'N_O_ref', 'N_Ni', 'N_Ni_ref', 'N_Al', 'N_Al_ref',
       'Al_modifier', 'Ni_modifier', 'E_i_DFT', 'E_i_EAM', 'modifier'],
      dtype='object')

In [36]:
df_resort["V_unscaled"] = df_resort["E_i_DFT"] - df_resort["E_i_EAM"]

/tmp/ipykernel_39994/1473106449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["V_unscaled"] = df_resort["E_i_DFT"] - df_resort["E_i_EAM"]


In [37]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

              E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM   E_ox  \
Species                                                                        
AlN-1           -969.413    -860.118     PureAl   -961.030   -860.160 -0.121   
AlN-2           -969.415    -859.881     PureAl   -961.030   -860.160 -0.121   
N                 -0.121       0.000       None        NaN        NaN -0.121   
N2               -16.489       0.000       None        NaN        NaN -0.121   
N2-short         -10.147       0.000       None        NaN        NaN -0.121   
N4-1             -20.387       0.000       None        NaN        NaN -0.121   
N4-2             -15.955       0.000       None        NaN        NaN -0.121   
N4-3             -31.099       0.000       None        NaN        NaN -0.121   
NN               -78.584       0.000       None        NaN        NaN -0.121   
NiAlN-1        -1314.924   -1120.123   PureNiAl  -1308.227  -1120.947 -0.121   
NiAlN-2        -1315.040   -1120.151   P

In [42]:
def V(e_conf_dft, modifier, e_ref_dft, E_O, N_O, E_ref_EAM, E_conf_EAM):
    v = e_conf_dft - modifier*e_ref_dft - (E_O* N_O)  -(modifier*E_ref_EAM - E_conf_EAM)
    return v

In [43]:
df_resort["V"] = V(df_resort["E_conf_DFT"], df_resort["modifier"], df_resort["E_ref_DFT"], df_resort["E_ox"], df_resort["N_O"],  df_resort["E_ref_EAM"], df_resort["E_conf_EAM"]) 

/tmp/ipykernel_39994/845931156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_resort["V"] = V(df_resort["E_conf_DFT"], df_resort["modifier"], df_resort["E_ref_DFT"], df_resort["E_ox"], df_resort["N_O"],  df_resort["E_ref_EAM"], df_resort["E_conf_EAM"])


In [44]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_resort)

              E_conf_DFT  E_conf_EAM ref_struct  E_ref_DFT  E_ref_EAM   E_ox  \
Species                                                                        
AlN-1           -969.413    -860.118     PureAl   -961.030   -860.160 -0.121   
AlN-2           -969.415    -859.881     PureAl   -961.030   -860.160 -0.121   
N                 -0.121       0.000       None        NaN        NaN -0.121   
N2               -16.489       0.000       None        NaN        NaN -0.121   
N2-short         -10.147       0.000       None        NaN        NaN -0.121   
N4-1             -20.387       0.000       None        NaN        NaN -0.121   
N4-2             -15.955       0.000       None        NaN        NaN -0.121   
N4-3             -31.099       0.000       None        NaN        NaN -0.121   
NN               -78.584       0.000       None        NaN        NaN -0.121   
NiAlN-1        -1314.924   -1120.123   PureNiAl  -1308.227  -1120.947 -0.121   
NiAlN-2        -1315.040   -1120.151   P

SyntaxError: invalid syntax (3374843745.py, line 1)